# SpareRoom


In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import folium
import json

<h1> London

<h2> Web Scraping housing info in London

In [7]:
def get_housings(url): 
    housings_list = list()
    import requests
    from bs4 import BeautifulSoup
    # Request for home page of NY housing info
    try:
        results_page = requests.get(url)
    except:
        return None
    try:
        soup = BeautifulSoup(results_page.content,'lxml')
    except:
        return None
    # Get all the feature housings and bold housings on the first page
    feature_housings = soup.find_all('article',class_='panel-listing-result listing-featured ')
    for housing in feature_housings:
        if housing.find('a'):
            detail_dict = dict()
            name = housing.find('h1').get_text()
            size = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").get_text()
            location = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").find('span', class_='listingLocation').get_text()
            detail_dict['Short_description'] = name
            # Split the room size and the location and store only size
            for i in range(1,len(size)):
                if size[i].isupper(): 
                    position = i
                    break
            detail_dict['Size'] = size[:position]
            detail_dict['Location'] = location
            link = 'https://www.spareroom.co.uk' + housing.find('a').get('href')
            housings_list.append([detail_dict,link])
    # Second Type of housing
    all_housings = soup.find_all('article', class_='panel-listing-result listing-bold ')
    for housing in all_housings:
        if housing.find('a'):
            detail_dict = dict()
            name = housing.find('h1').get_text()
            size = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").get_text()
            location = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").find('span', class_='listingLocation').get_text()
            detail_dict['Short_description'] = name
            # Split the room size and the location and store only size
            for i in range(1,len(size)):
                if size[i].isupper(): 
                    position = i
                    break
            detail_dict['Size'] = size[:position]
            detail_dict['Location'] = location
            link = 'https://www.spareroom.co.uk' + housing.find('a').get('href')
            housings_list.append([detail_dict,link])
    # Third type of housing:
    early_bird_housings = soup.find_all('article', class_='panel-listing-result listing-free ')
    for housing in early_bird_housings:
        if housing.find('a'):
            detail_dict = dict()
            name = housing.find('h1').get_text()
            size = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").get_text()
            detail_dict['Short_description'] = name
            location = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").find('span', class_='listingLocation').get_text()
            # Split the room size and the location and store only size
            for i in range(1,len(size)):
                if size[i].isupper(): 
                    position = i
                    break
            detail_dict['Size'] = size[:position]
            detail_dict['Location'] = location
            link = 'https://www.spareroom.co.uk' + housing.find('a').get('href')
            housings_list.append([detail_dict,link])
#     Turn to next page and get more housings
    if soup.find('ul',class_='navnext'):
        if soup.find('ul',class_='navnext').find('a'):
            next_page_token = soup.find('ul',class_='navnext').find('a').get('href')
            next_page = 'https://www.spareroom.co.uk/flatshare/' + next_page_token
        # Use recursion to get all the attractions    
            housings_list.extend(get_housings(next_page))
    return housings_list


In [8]:
url_london = 'https://www.spareroom.co.uk/flatshare/?search_id=745817425&'
list_ = get_housings(url_london)
print(f'We have find {len(list_)} results for you.')

We have find 1100 results for you.


In [9]:
def get_details(list_housings):
    count = 0 
    full_dict = dict()
    for dict_info, link in list_housings:
        import requests
        from bs4 import BeautifulSoup
        # Request for details of each housing in NY
        try:
            results_page = requests.get(link)
        except:
            continue
        try:
            soup = BeautifulSoup(results_page.content,'lxml')
        except:
            continue
        # Get housing price for each room 
        price_list = list()
        if soup.find('ul', class_='room-list'): 
            prices = soup.find('ul', class_='room-list')
            price = prices.find_all('strong', class_='room-list__price')
            for price_i in price:
                price_list.append(price_i.get_text())
        # price 这里似乎有个bug，但是我暂时懒得改
        if soup.find('h3', class_='feature__heading'):
            price_list.append(soup.find('h3', class_='feature__heading').get_text())
        # Clean the data to remove 'Availability' in the price list
        if 'Availability' in price_list:
            price_list.remove('Availability')
        dict_info['Price'] = price_list
        # Get Availability for each housing
        avail = soup.find('section', class_='feature feature--availability')
        if avail:
            all_info_avail = avail.find_all('dd', class_='feature-list__value')
            availablility = all_info_avail[0].get_text()      
            min_period = all_info_avail[1].get_text()      
            max_period = all_info_avail[2].get_text() 
            dict_info['Availablility'] = availablility
            dict_info['Min_period'] = min_period
            dict_info['Max_period'] = max_period
        # Get Amenities info for each housing
        amenities = soup.find('section', class_='feature feature--amenities')
        if amenities:
            all_info_amenities = amenities.find_all('dd', class_='feature-list__value')
            if all_info_amenities:
                furnish = all_info_amenities[0].get_text()
                dict_info['Furnishing'] = furnish
        else:
            dict_info['Furnishing'] = 'No info'
        # Female/Male requirement
        preference = soup.find('section', class_='feature feature--household-preferences')
        if preference:
            preference_info = preference.find_all('dt', class_='feature-list__key')
            preference_detail = preference.find_all('dd', class_='feature-list__value')
            if preference_info and preference_detail:
                for info in preference_info:
                    if info.get_text() == 'Gender':
                        info_index = preference_info.index(info)
                        break
                try:
                    gender = preference_detail[info_index].get_text() 
                    dict_info['Gender_requirement'] = gender
                except:
                    dict_info['Gender_requirement'] = 'No info'
        full_dict[count] = dict_info
        count = count + 1
    return full_dict
        

In [10]:
list_housings = list_
dict_ = get_details(list_housings)

<h3> Clean the data

In [12]:
for key in dict_.keys():
    for subkey in dict_[key].keys():
        if isinstance(dict_[key][subkey], str):
            dict_[key][subkey] = dict_[key][subkey].lstrip().rstrip()
# Clean the price list
for key in dict_.keys():
    list_in = list()
    for item in list(dict_[key]['Price']):
        list_in.append(item.strip('\n'))
    dict_[key]['Price'] = list_in

<h3> Create a dataframe to store the info

In [13]:
df_info = pd.DataFrame(dict_)
df_info = df_info.T

# Fix furnished problem
for i in df_info.index:
    if df_info.iloc[i]['Furnishing'] in ['Furnished', 'Part Furnished', 'Unfurnished']:
        pass
    else:
        df_info.iloc[i]['Furnishing'] = 'No info'
# Fix Gender_requirement problem
for i in df_info.index:
    if not df_info.iloc[i]['Gender_requirement'] in ['Males or females', 'Female preferred', 'No info',
       'Male preferred']:
        df_info.iloc[i]['Gender_requirement'] = 'No info'

In [14]:
# Create a new col in df_info to store average price for a single person
pattern = r'\d+'  
price_per_month = list()

for i in df_info.index:
    list_in_price = list()
    for item in df_info.iloc[i]['Price']:
        item = item.replace(',', '')
        if 'pw' in item:
            price_info = int((re.findall(pattern, item))[0])
            list_in_price.append(price_info*4)
        if 'pcm' in item:
            price_info = int((re.findall(pattern, item))[0])
            list_in_price.append(price_info)
    price_per_month.append(list_in_price)
price_per_month = np.array(price_per_month)
df_info['Price_per_month'] = price_per_month
list_avg_price = list()
for i in df_info.index:
    list_avg_price.append(np.mean(df_info.iloc[i]['Price_per_month']) )
df_info['Average_price_per_month'] = np.array(list_avg_price)
# Create a new col in df_info to store zip code for each housing


/usr/local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [15]:
# Create a new col for zip code
list_pos = list()
for i in df_info.index:
    for s in df_info.iloc[i]['Location']:
        if s == '(':
            position = df_info.iloc[i]['Location'].index(s)
            break
    list_pos.append(df_info.iloc[i]['Location'][position:].strip('(').strip(')'))
df_info['Postal_code'] = np.array(list_pos)

In [16]:
df_info

,Availablility,Furnishing,Gender_requirement,Location,Max_period,Min_period,Price,Short_description,Size,Price_per_month,Average_price_per_month,Postal_code
0,27 Nov 2018,Furnished,Males or females,Colliers Wood (SW19),None,None,"[£600 pcm, £600 pcm, £950 pcm, £600 pcm, £650 ...","Own Space, Clean Space, Professional Sharers",Double room,"[600, 600, 950, 600, 650, 650, 950]",714.285714,SW19
1,02 Feb 2019,Furnished,Female preferred,Anerley (SE20),None,12 months,[£575 pcm],Stylish Huge House - Amazing Double Available,Double room,[575],575.000000,SE20
2,Now,Unfurnished,No info,Acton (W3),None,6 months,"[£1,200 pcm (whole property)]",Fully refurbished 1 bedroom flat in Acton.,1 bed flat,[1200],1200.000000,W3
3,Now,Furnished,Males or females,Canonbury (N1),None,None,[£210 pw],Canonbury Square. Great double bedroom,Double room,[840],840.000000,N1
4,Now,Furnished,No info,Old Street (N1),None,6 months,[£625 pw (whole property)],Loft style 2 bed apartment in Old Street.,2 bed flat,[2500],2500.000000,N1
5,Now,Furnished,Males or females,Canary Wharf (E14),None,6 months,"[£925 pcm, £1,025 pcm, £1,100 pcm]","Exclusive New Riverside Flat, X3 Spacious Rooms",Double room,"[925, 1025, 1100]",1016.666667,E14
6,19 Dec 2018,Furnished,No info,Finsbury Park (N4),None,6 months,"[£1,100 pcm (whole property)]",Self Contained New Studio All Bills Included,1 bed flat,[1100],1100.000000,N4
7,30 Nov 2018,Furnished,No info,Surrey Quays (SE16),None,6 months,[£311 pw (whole property)],"[Large Houseboat, Surrey Quays]",1 bed property,[1244],1244.000000,SE16
8,Now,Furnished,Males or females,St John's Wood (NW8),None,3 months,[£210 pw],Splendid Double Room Besides Regent's Park,Double room,[840],840.000000,NW8
9,Now,Furnished,Males or females,Camden (NW1),None,None,"[£225 pw, £230 pw]",Camden Town- Great NEW house!,2 doubles,"[900, 920]",910.000000,NW1


<h4> Create a function for the above

In [ ]:
# Create a function for the above:
def clean(dict_):
    for key in dict_.keys():
        for subkey in dict_[key].keys():
            if isinstance(dict_[key][subkey], str):
                dict_[key][subkey] = dict_[key][subkey].lstrip().rstrip()
    # Clean the price list
    for key in dict_.keys():
        list_in = list()
        for item in list(dict_[key]['Price']):
            list_in.append(item.strip('\n'))
        dict_[key]['Price'] = list_in
        
    df_info = pd.DataFrame(dict_)
    df_info = df_info.T

    # Fix furnished problem
    for i in df_info.index:
        if df_info.iloc[i]['Furnishing'] in ['Furnished', 'Part Furnished', 'Unfurnished']:
            pass
        else:
            df_info.iloc[i]['Furnishing'] = 'No info'
    # Fix Gender_requirement problem
    for i in df_info.index:
        if df_info.iloc[i]['Gender_requirement'] in ['Males or females', 'Female preferred', 'No info','Male preferred']:
            pass
        else:
            df_info.iloc[i]['Gender_requirement'] = 'No info'
    # Create a new col in df_info to store average price for a single person
    pattern = r'\d+'  
    price_per_month = list()

    for i in df_info.index:
        list_in_price = list()
        for item in df_info.iloc[i]['Price']:
            item = item.replace(',', '')
            if 'pw' in item:
                price_info = int((re.findall(pattern, item))[0])
                list_in_price.append(price_info*4)
            if 'pcm' in item:
                price_info = int((re.findall(pattern, item))[0])
                list_in_price.append(price_info)
        price_per_month.append(list_in_price)
    price_per_month = np.array(price_per_month)
    df_info['Price_per_month'] = price_per_month
    list_avg_price = list()
    for i in df_info.index:
        list_avg_price.append(np.mean(df_info.iloc[i]['Price_per_month']) )
    df_info['Average_price_per_month'] = np.array(list_avg_price)
    # Create a new col in df_info to store zip code for each housing
    # Create a new col for zip code
    list_pos = list()
    for i in df_info.index:
        for s in df_info.iloc[i]['Location']:
            if s == '(':
                position = df_info.iloc[i]['Location'].index(s)
                break
        list_pos.append(df_info.iloc[i]['Location'][position:].strip('(').strip(')'))
    df_info['Postal_code'] = np.array(list_pos)
    return df_info
      

<h3> Data Visualization

In [22]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn')# seaborn is a style that have different colors

<h4> To see if there is a gender preference in London

In [27]:
gender_group = df_info.groupby('Gender_requirement')
gender_group.size().plot(kind='bar', title='Gender requirement for rent in London')

<h4> To see the housing condition in postal code area in London

In [28]:
Postal_code_group = df_info.groupby('Postal_code')
Postal_code_group.size().plot(kind='bar', title='Postal code area in London', figsize=(24,24))

<h4> To see if there is more furnished apartment in London

In [ ]:
furnish_group = df_info.groupby('Furnishing')
furnish_group.size().plot(kind='bar', title='Furnishing status of housing in London')

<h4> To see the size distribution in London

In [ ]:
size_group = df_info.groupby('Size')
size_group.size().plot(kind='bar', title='Housing size in London')

<h4> To see the average per-month price change

In [ ]:
df_info['Average_price_per_month'].plot(kind='line', title='Average price in London')

In [ ]:
# Group the average price
def GroupColFunc(df, ind, col):
    if df[col].loc[ind] < 500:
        return 'Lower than 500'
    elif 1000 > df[col].loc[ind] >= 500:
        return 'Between 500 and 1000'
    elif 1500 > df[col].loc[ind] >= 1000:
        return 'Between 1000 and 1500'
    else:
        return 'Above 1500'
avg_price_group = df_info.groupby(lambda x: GroupColFunc(df_info, x, 'Average_price_per_month')) 
avg_price_group.size().plot(kind='bar', title='Housing average price in London')
# Add a col of average price group for further analysis
avg_price_list = list()
for i in df_info.index:
    avg_price_list.append(GroupColFunc(df_info, i, 'Average_price_per_month'))
df_info['Avg_price_group'] = np.array(avg_price_list)

In [ ]:
# Group by both size and average price
size_avg_groups = df_info.groupby(['Size','Avg_price_group']).size().unstack()

In [ ]:
COL_NUM = 2
ROW_NUM = 2
import matplotlib.pyplot as plt
fig, axes = plt.subplots(ROW_NUM, COL_NUM, figsize=(12,12))

for i, (avg, size) in enumerate(size_avg_groups.items()): 
    ax = axes[int(i/COL_NUM), i%COL_NUM]
    size = size.sort_values(ascending=False) # sort the data in descending order 
    size.plot(kind='barh', ax=ax)
    ax.set_title(avg)

plt.tight_layout() 

<h3> Heat Map

In [33]:
m = folium.Map(location = [51.5074, 0.1278],zoom_start=10)

In [34]:
zip_groups = df_info.groupby('Postal_code')
sizes = pd.DataFrame(zip_groups.size())
sizes.rename(columns={0:"size"},inplace=True)# Rename the col besides index to 'size'
sizes.reset_index(level=0, inplace=True)# level 0 here means no index,the index becomes row number
sizes['Zip'] = sizes['Postal_code']
sizes.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 3 columns):
Postal_code    109 non-null object
size           109 non-null int64
Zip            109 non-null object
dtypes: int64(1), object(2)
memory usage: 2.6+ KB


In [35]:
# read the geojson file and get all the postcode
with open("london_postcodes.geojson",'r') as load_f:
    load_dict = json.load(load_f)
full_pos = list()
for item in load_dict['features']:
    full_pos.append(item['properties']['Name'])


In [36]:
df_zip = pd.DataFrame(np.array([full_pos])).T
df_zip = df_zip.rename(columns={0:'Zip_code'})

In [37]:
zip_size = list()
for i in df_zip.index:
    for j in sizes.index:
        if df_zip.iloc[i]['Zip_code'] == sizes.iloc[j]['Zip']:
            zip_size.append(sizes.iloc[j]['size'])
            break
        elif j == (len(sizes) - 1):
            zip_size.append(0)
df_zip['size'] = np.array(zip_size)  

In [39]:
m.choropleth(geo_data='london_postcodes.geojson', data=df_zip, 
             columns=['Zip_code','size'],
             key_on='feature.properties.Name',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.8,
             legend_name='Distribution of housing')
folium.LayerControl().add_to(m)

/usr/local/lib/python3.6/site-packages/folium/folium.py:432: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [40]:
m

<h4> Create a function for the above

In [ ]:
# a function to use google api to get the lat and lng for a location
def get_lat_lng(address_string,api_key):
    url=f"https://maps.googleapis.com/maps/api/geocode/json?address={address_string}&key={api_key}" 
    import requests
    response = requests.get(url)
    if response.status_code != 200:
        print('Something went wrong about requests due to internet problem or typing error.')
    else:
        try:
            response_ = response.json()
        except:
            print("Sth wrong with json decoding process")
        return [response_['results'][0]['geometry']['location']['lat'],response_['results'][0]['geometry']['location']['lng']]

In [ ]:
# All_above in one function 
def map_rent(location, df_info, api_key):
    location = location.lower() 
    api_key = api_key
    m = folium.Map(location = get_lat_lng(location,api_key),zoom_start=10)
    zip_groups = df_info.groupby('Postal_code')
    sizes = pd.DataFrame(zip_groups.size())
    sizes.rename(columns={0:"size"},inplace=True)# Rename the col besides index to 'size'
    sizes.reset_index(level=0, inplace=True)# level 0 here means no index,the index becomes row number
    sizes['Zip'] = sizes['Postal_code']
    # read the geojson file and get all the postcode
    with open(f"{location}_postcodes.geojson",'r') as load_f:
        load_dict = json.load(load_f)
    full_pos = list()
    for item in load_dict['features']:
        full_pos.append(item['properties']['Name'])
    df_zip = pd.DataFrame(np.array([full_pos])).T
    df_zip = df_zip.rename(columns={0:'Zip_code'})
    zip_size = list()
    for i in df_zip.index:
        for j in sizes.index:
            if df_zip.iloc[i]['Zip_code'] == sizes.iloc[j]['Zip']:
                zip_size.append(sizes.iloc[j]['size'])
                break
            elif j == (len(sizes) - 1):
                zip_size.append(0)
    df_zip['size'] = np.array(zip_size) 
    m.choropleth(geo_data=f'{location}_postcodes.geojson', data=df_zip, 
             columns=['Zip_code','size'],
             key_on='feature.properties.Name',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.8,
             legend_name='Distribution of housing')
    folium.LayerControl().add_to(m)
    return m

    

<h1> Liverpool

In [12]:
def get_housings(url): 
    housings_list = list()
    import requests
    from bs4 import BeautifulSoup
    # Request for home page of NY housing info
    try:
        results_page = requests.get(url)
    except:
        return None
    try:
        soup = BeautifulSoup(results_page.content,'lxml')
    except:
        return None
    # Get all the feature housings and bold housings on the first page
    feature_housings = soup.find_all('article',class_='panel-listing-result listing-featured ')
    for housing in feature_housings:
        if housing.find('a'):
            detail_dict = dict()
            name = housing.find('h1').get_text()
            size = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").get_text()
            location = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").find('span', class_='listingLocation').get_text()
            detail_dict['Short_description'] = name
            # Split the room size and the location and store only size
            for i in range(1,len(size)):
                if size[i].isupper(): 
                    position = i
                    break
            detail_dict['Size'] = size[:position]
            detail_dict['Location'] = location
            link = 'https://www.spareroom.co.uk' + housing.find('a').get('href')
        housings_list.append([detail_dict,link])
    # Second type
    all_housings = soup.find_all('article', class_='panel-listing-result listing-bold ')
    for housing in all_housings:
        if housing.find('a'):
            detail_dict = dict()
            name = housing.find('h1').get_text()
            size = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").get_text()
            location = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").find('span', class_='listingLocation').get_text()
            detail_dict['Short_description'] = name
            # Split the room size and the location and store only size
            for i in range(1,len(size)):
                if size[i].isupper(): 
                    position = i
                    break
            detail_dict['Size'] = size[:position]
            detail_dict['Location'] = location
            link = 'https://www.spareroom.co.uk' + housing.find('a').get('href')
        housings_list.append([detail_dict,link])
    # Third type
    early_bird_housings = soup.find_all('article', class_='panel-listing-result listing-free ')
    for housing in early_bird_housings:
        if housing.find('a'):
            detail_dict = dict()
            name = housing.find('h1').get_text()
            size = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").get_text()
            detail_dict['Short_description'] = name
            location = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").find('span', class_='listingLocation').get_text()
            # Split the room size and the location and store only size
            for i in range(1,len(size)):
                if size[i].isupper(): 
                    position = i
                    break
            detail_dict['Size'] = size[:position]
            detail_dict['Location'] = location
            link = 'https://www.spareroom.co.uk' + housing.find('a').get('href')
        housings_list.append([detail_dict,link])
#     Turn to next page and get more housings
    if soup.find('ul',class_='navnext'):
        if soup.find('ul',class_='navnext').find('a'):
            next_page_token = soup.find('ul',class_='navnext').find('a').get('href')
            next_page = 'https://www.spareroom.co.uk/flatshare/' + next_page_token
        # Use recursion to get all the attractions    
            housings_list.extend(get_housings(next_page))
    return housings_list



In [13]:
url_liver = 'https://www.spareroom.co.uk/flatshare/?search_id=746073457&'
list_liver = get_housings(url_liver)
print(f'We have find {len(list_liver)} results for you.')

We have find 732 results for you.


In [ ]:
list_liver

In [ ]:
def get_details(list_housings):
    count = 0 
    full_dict = dict()
    for dict_info, link in list_housings:
        import requests
        from bs4 import BeautifulSoup
        # Request for details of each housing in NY
        try:
            results_page = requests.get(link)
        except:
            continue
        try:
            soup = BeautifulSoup(results_page.content,'lxml')
        except:
            continue
        # Get housing price for each room 
        price_list = list()
        if soup.find('ul', class_='room-list'): 
            prices = soup.find('ul', class_='room-list')
            price = prices.find_all('strong', class_='room-list__price')
            for price_i in price:
                price_list.append(price_i.get_text())
        # price 这里似乎有个bug，但是我暂时懒得改
        if soup.find('h3', class_='feature__heading'):
            price_list.append(soup.find('h3', class_='feature__heading').get_text())
        # Clean the data to remove 'Availability' in the price list
        if 'Availability' in price_list:
            price_list.remove('Availability')
        dict_info['Price'] = price_list
        # Get Availability for each housing
        avail = soup.find('section', class_='feature feature--availability')
        if avail:
            all_info_avail = avail.find_all('dd', class_='feature-list__value')
            availablility = all_info_avail[0].get_text()      
            min_period = all_info_avail[1].get_text()      
            max_period = all_info_avail[2].get_text() 
            dict_info['Availablility'] = availablility
            dict_info['Min_period'] = min_period
            dict_info['Max_period'] = max_period
        # Get Amenities info for each housing
        amenities = soup.find('section', class_='feature feature--amenities')
        if amenities:
            all_info_amenities = amenities.find_all('dd', class_='feature-list__value')
            if all_info_amenities:
                furnish = all_info_amenities[0].get_text()
                dict_info['Furnishing'] = furnish
        else:
            dict_info['Furnishing'] = 'No info'
        # Female/Male requirement
        preference = soup.find('section', class_='feature feature--household-preferences')
        if preference:
            preference_info = preference.find_all('dt', class_='feature-list__key')
            preference_detail = preference.find_all('dd', class_='feature-list__value')
            if preference_info and preference_detail:
                for info in preference_info:
                    if info.get_text() == 'Gender':
                        info_index = preference_info.index(info)
                        break
                try:
                    gender = preference_detail[info_index].get_text() 
                    dict_info['Gender_requirement'] = gender
                except:
                    dict_info['Gender_requirement'] = 'No info'
        full_dict[count] = dict_info
        count = count + 1
    return full_dict
        

In [ ]:
list_housings_liver = list_liver
dict_liver = get_details(list_housings_liver)

In [ ]:
dict_liver

<h3>Clean the data

In [ ]:
for key in dict_liver.keys():
    for subkey in dict_liver[key].keys():
        if isinstance(dict_liver[key][subkey], str):
            dict_liver[key][subkey] = dict_liver[key][subkey].lstrip().rstrip()
# Clean the price list
for key in dict_liver.keys():
    list_in = list()
    for item in list(dict_liver[key]['Price']):
        list_in.append(item.strip('\n'))
    dict_liver[key]['Price'] = list_in

<h3>Create a dataframe to store the info

In [ ]:
df_info_liver = pd.DataFrame(dict_liver)
df_info_liver = df_info_liver.T

# Fix furnished problem
for i in df_info_liver.index:
    if df_info_liver.iloc[i]['Furnishing'] in ['Furnished', 'Part Furnished', 'Unfurnished']:
        pass
    else:
        df_info_liver.iloc[i]['Furnishing'] = 'No info'
# Fix Gender_requirement problem
for i in df_info_liver.index:
    if not df_info_liver.iloc[i]['Gender_requirement'] in ['Males or females', 'Female preferred', 'No info',
       'Male preferred']:
        df_info_liver.iloc[i]['Gender_requirement'] = 'No info'

In [ ]:
# Create a new col in df_info_liver to store average price for a single person
pattern = r'\d+'  
price_per_month = list()

for i in df_info_liver.index:
    list_in_price = list()
    for item in df_info_liver.iloc[i]['Price']:
        item = item.replace(',', '')
        if 'pw' in item:
            price_info = int((re.findall(pattern, item))[0])
            list_in_price.append(price_info*4)
        if 'pcm' in item:
            price_info = int((re.findall(pattern, item))[0])
            list_in_price.append(price_info)
    price_per_month.append(list_in_price)
price_per_month = np.array(price_per_month)
df_info_liver['Price_per_month'] = price_per_month
list_avg_price = list()
for i in df_info_liver.index:
    list_avg_price.append(np.mean(df_info_liver.iloc[i]['Price_per_month']) )
df_info_liver['Average_price_per_month'] = np.array(list_avg_price)
# Create a new col in df_info_liver to store zip code for each housing




In [ ]:
df_info_liver

In [ ]:
# Create a new col for zip code
list_pos = list()
for i in df_info_liver.index:
    for s in df_info.iloc[i]['Location']:
        if s == '(':
            position =df_info_liver.iloc[i]['Location'].index(s)
            break
    list_pos.append(df_info_liver.iloc[i]['Location'][position:].strip('(').strip(')'))
df_info_liver['Postal_code'] = np.array(list_pos)

<h3>Data Visualization

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn')# seaborn is a style that have different colors

<h3>To see if there is a gender preference in Liverpool

In [ ]:
gender_group_liver = df_info_liver.groupby('Gender_requirement')
gender_group_liver.size().plot(kind='bar', title='Gender requirement for rent in Liverpool')

<h3>To see if there is more furnished apartment in Liverpool

In [ ]:
furnish_group_liver = df_info_liver.groupby('Furnishing')
furnish_group_liver.size().plot(kind='bar', title='Furnishing status of housing in Liverpool')

<h3>To see the size distribution in Liverpool 

In [ ]:
size_group_liver = df_info_liver.groupby('Size')
size_group_liver.size().plot(kind='bar', title='Housing size in Liverpool')

<h3>To see the average per-month price change

In [ ]:
df_info_liver['Average_price_per_month'].plot(kind='line', title='Average price in Liverpool')

In [ ]:
# Group the average price
def GroupColFunc(df, ind, col):
    if df[col].loc[ind] < 500:
        return 'Lower than 500'
    elif 1000 > df[col].loc[ind] >= 500:
        return 'Between 500 and 1000'
    else:
        return 'Above 1000'
avg_price_group_liver = df_info_liver.groupby(lambda x: GroupColFunc(df_info, x, 'Average_price_per_month')) 
avg_price_group_liver.size().plot(kind='bar', title='Housing average price in London')
# Add a col of average price group for further analysis
avg_price_list = list()
for i in df_info_liver.index:
    avg_price_list.append(GroupColFunc(df_info, i, 'Average_price_per_month'))
df_info_liver['Avg_price_group'] = np.array(avg_price_list)

In [ ]:
# Group by both size and average price
size_avg_groups_liver = df_info_liver.groupby(['Size','Avg_price_group']).size().unstack()

In [ ]:
COL_NUM = 2
ROW_NUM = 2
import matplotlib.pyplot as plt
fig, axes = plt.subplots(ROW_NUM, COL_NUM, figsize=(12,12))

for i, (avg, size) in enumerate(size_avg_groups_liver.items()): 
    ax = axes[int(i/COL_NUM), i%COL_NUM]
    size = size.sort_values(ascending=False) # sort the data in descending order 
    size.plot(kind='barh', ax=ax)
    ax.set_title(avg)

plt.tight_layout() 

<h2>Heat Map

In [ ]:
m = folium.Map(location = [53.3, -3],zoom_start=10)

In [ ]:
m

In [ ]:
zip_groups = df_info_liver.groupby('Postal_code')
sizes = pd.DataFrame(zip_groups.size())
sizes.rename(columns={0:"size"},inplace=True)# Rename the col besides index to 'size'
sizes.reset_index(level=0, inplace=True)# level 0 here means no index,the index becomes row number
sizes['Zip'] = sizes['Postal_code']
sizes.info()

In [ ]:
sizes

In [ ]:
# read the geojson file and get all the postcode
with open("liverpool_postcodes.geojson",'r') as load_f:
    load_dict = json.load(load_f)

full_pos = list()
for item in load_dict['features']:
    full_pos.append(item['properties']['Name'])



In [ ]:
df_zip = pd.DataFrame(np.array([full_pos])).T
df_zip = df_zip.rename(columns={0:'Zip_code'})

In [ ]:
zip_size = list()
for i in df_zip.index:
    for j in sizes.index:
        if df_zip.iloc[i]['Zip_code'] == sizes.iloc[j]['Zip']:
            zip_size.append(sizes.iloc[j]['size'])
            break
        elif j == (len(sizes) - 1):
            zip_size.append(0)
df_zip['size'] = np.array(zip_size)   

In [ ]:
m.choropleth(geo_data='liverpool_postcodes.geojson', data=df_zip, 
             columns=['Zip_code','size'],
             key_on='feature.properties.name',
             fill_color='GnBu', fill_opacity=0.7, line_opacity=0.8,
             legend_name='Distribution of Incidents')
folium.LayerControl().add_to(m)

In [ ]:
m

<h1> Leeds

In [18]:
def get_housings(url): 
    housings_list = list()
    import requests
    from bs4 import BeautifulSoup
    # Request for home page of NY housing info
    try:
        results_page = requests.get(url)
    except:
        return None
    try:
        soup = BeautifulSoup(results_page.content,'lxml')
    except:
        return None
    # Get all the feature housings and bold housings on the first page
    feature_housings = soup.find_all('article',class_='panel-listing-result listing-featured ')
    for housing in feature_housings:
        if housing.find('a'):
            detail_dict = dict()
            name = housing.find('h1').get_text()
            size = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").get_text()
            location = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").find('span', class_='listingLocation').get_text()
            detail_dict['Short_description'] = name
            # Split the room size and the location and store only size
            for i in range(1,len(size)):
                if size[i].isupper(): 
                    position = i
                    break
            detail_dict['Size'] = size[:position]
            detail_dict['Location'] = location
            link = 'https://www.spareroom.co.uk' + housing.find('a').get('href')
        housings_list.append([detail_dict,link])
    # Second type
    all_housings = soup.find_all('article', class_='panel-listing-result listing-bold ')
    for housing in all_housings:
        if housing.find('a'):
            detail_dict = dict()
            name = housing.find('h1').get_text()
            size = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").get_text()
            location = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").find('span', class_='listingLocation').get_text()
            detail_dict['Short_description'] = name
            # Split the room size and the location and store only size
            for i in range(1,len(size)):
                if size[i].isupper(): 
                    position = i
                    break
            detail_dict['Size'] = size[:position]
            detail_dict['Location'] = location
            link = 'https://www.spareroom.co.uk' + housing.find('a').get('href')
        housings_list.append([detail_dict,link])
    # Third type
    early_bird_housings = soup.find_all('article', class_='panel-listing-result listing-free ')
    for housing in early_bird_housings:
        if housing.find('a'):
            detail_dict = dict()
            name = housing.find('h1').get_text()
            size = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").get_text()
            detail_dict['Short_description'] = name
            location = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").find('span', class_='listingLocation').get_text()
            # Split the room size and the location and store only size
            for i in range(1,len(size)):
                if size[i].isupper(): 
                    position = i
                    break
            detail_dict['Size'] = size[:position]
            detail_dict['Location'] = location
            link = 'https://www.spareroom.co.uk' + housing.find('a').get('href')
        housings_list.append([detail_dict,link])
    #     Turn to next page and get more housings
        if soup.find('ul',class_='navnext'):
            if soup.find('ul',class_='navnext').find('a'):
                next_page_token = soup.find('ul',class_='navnext').find('a').get('href')
                next_page = 'https://www.spareroom.co.uk/flatshare/' + next_page_token
            # Use recursion to get all the attractions    
                housings_list.extend(get_housings(next_page))
    return housings_list


In [19]:
url_leeds = 'https://www.spareroom.co.uk/flatshare/?search_id=748742143&'
list_leeds = get_housings(url_leeds)
print(f'We have find {len(list_leeds)} results for you.')

We have find 1348 results for you.


In [ ]:
def get_details(list_leeds):
    count = 0 
    full_dict = dict()
    for dict_info, link in list_leeds:
        import requests
        from bs4 import BeautifulSoup
        # Request for details of each housing in NY
        try:
            results_page = requests.get(link)
        except:
            continue
        try:
            soup = BeautifulSoup(results_page.content,'lxml')
        except:
            continue
        # Get housing price for each room 
        price_list = list()
        if soup.find('ul', class_='room-list'): 
            prices = soup.find('ul', class_='room-list')
            price = prices.find_all('strong', class_='room-list__price')
            for price_i in price:
                price_list.append(price_i.get_text())
        if soup.find('h3', class_='feature_heading')
            prices = soup.find_all('h3', class_='feature__heading')
            print(prices)
            for price_i in prices:
                price_list.append(price_i.get_text()[0])
        # Clean the data to remove 'Availability' in the price list
#         if 'Availability' in price_list:
#             price_list.remove('Availability')
#         dict_info['Price'] = price_list
#         # Get Availability for each housing
#         avail = soup.find('section', class_='feature feature--availability')
#         if avail:
#             all_info_avail = avail.find_all('dd', class_='feature-list__value')
#             availablility = all_info_avail[0].get_text()      
#             min_period = all_info_avail[1].get_text()      
#             max_period = all_info_avail[2].get_text() 
#             dict_info['Availablility'] = availablility
#             dict_info['Min_period'] = min_period
#             dict_info['Max_period'] = max_period
#         # Get Amenities info for each housing
#         amenities = soup.find('section', class_='feature feature--amenities')
#         if amenities:
#             all_info_amenities = amenities.find_all('dd', class_='feature-list__value')
#             if all_info_amenities:
#                 furnish = all_info_amenities[0].get_text()
#                 dict_info['Furnishing'] = furnish
#         else:
#             dict_info['Furnishing'] = 'No info'
#         # Female/Male requirement
#         preference = soup.find('section', class_='feature feature--household-preferences')
#         if preference:
#             preference_info = preference.find_all('dt', class_='feature-list__key')
#             preference_detail = preference.find_all('dd', class_='feature-list__value')
#             if preference_info and preference_detail:
#                 for info in preference_info:
#                     if info.get_text() == 'Gender':
#                         info_index = preference_info.index(info)
#                         break
#                 try:
#                     gender = preference_detail[info_index].get_text() 
#                     dict_info['Gender_requirement'] = gender
#                 except:
#                     dict_info['Gender_requirement'] = 'No info'
#         full_dict[count] = dict_info
#         count = count + 1
    
#     return full_dict

In [23]:
price_list = get_details(list_leeds)

In [24]:
price_list

['£285 pcm',
 'Availability',
 'Extra cost',
 'Amenities',
 'Current household',
 'New housemate preferences']

In [16]:
dict_leeds = get_details(list_leeds)

<h1> Manchester

<h1> Create a blueprint for all the four cities

In [ ]:
class rental:
    def __init__(self, url, location):
        self.location = location
        self.url = url
        self.dict_ = get_details(get_housings(self.url))
    @property
    def df(self):
        return clean(self.dict_)
    def graph(self, group):
        # only for simple group function, cannot be used for graph with two groupings. 
        # group here is for the name of one column in the df that you want to use groupby function.
        %matplotlib inline
        import matplotlib.pyplot as plt
        plt.style.use('seaborn')
        self.group = self.df.groupby(str(group))
        return self.group.size().plot(kind='bar', title=f'{group} for rent in London')
    def heat_map(self):
        return map_rent(self.location, self.df)

In [ ]:
london = rental('https://www.spareroom.co.uk/flatshare/?search_id=745817425&', 'london')

In [ ]:
london.df

In [ ]:
london.graph('Size')

In [ ]:
london.heat_map()